<a href="https://colab.research.google.com/github/gupta-keshav/age_gender_classification/blob/master/alignedGenderSubmit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%pip install patool
import patoolib
patoolib.extract_archive("/content/drive/My Drive/Adience/alignedGender.rar", outdir="/content")

     |████████████████████████████████| 81kB 4.0MB/s 
patool: Extracting /content/drive/My Drive/Adience/alignedGender.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/Adience/alignedGender.rar"
patool:     with cwd='/content'
patool: ... /content/drive/My Drive/Adience/alignedGender.rar extracted to `/content'.


'/content'

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, PReLU, ReLU
from keras import optimizers, regularizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
np.random.seed(48)
from keras.applications import vgg19
from keras.models import Model
input_shape = (224, 224, 3)
vgg = vgg19.VGG19(include_top=False ,weights='imagenet', 
                                     input_shape=input_shape)   

In [0]:
last = vgg.output

In [15]:
x = Flatten()(last)
x = Dropout(0.3)(x)
x = Dense(4096, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(x)
x = Dropout(0.55)(x)

In [0]:
preds = Dense(1, activation = 'sigmoid')(x)

In [0]:
model = Model(vgg.input, preds)

In [18]:
import pandas as pd
layers = [(layer.output, layer.name, layer.trainable) for layer in model.layers]
pd.set_option('max_colwidth', -1)

pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,"Tensor(""input_2:0"", shape=(?, 224, 224, 3), dtype=float32)",input_2,False
1,"Tensor(""block1_conv1_1/Relu:0"", shape=(?, 224, 224, 64), dtype=float32)",block1_conv1,True
2,"Tensor(""block1_conv2_1/Relu:0"", shape=(?, 224, 224, 64), dtype=float32)",block1_conv2,True
3,"Tensor(""block1_pool_1/MaxPool:0"", shape=(?, 112, 112, 64), dtype=float32)",block1_pool,True
4,"Tensor(""block2_conv1_1/Relu:0"", shape=(?, 112, 112, 128), dtype=float32)",block2_conv1,True
5,"Tensor(""block2_conv2_1/Relu:0"", shape=(?, 112, 112, 128), dtype=float32)",block2_conv2,True
6,"Tensor(""block2_pool_1/MaxPool:0"", shape=(?, 56, 56, 128), dtype=float32)",block2_pool,True
7,"Tensor(""block3_conv1_1/Relu:0"", shape=(?, 56, 56, 256), dtype=float32)",block3_conv1,True
8,"Tensor(""block3_conv2_1/Relu:0"", shape=(?, 56, 56, 256), dtype=float32)",block3_conv2,True
9,"Tensor(""block3_conv3_1/Relu:0"", shape=(?, 56, 56, 256), dtype=float32)",block3_conv3,True


In [0]:
for i in range(17):
  model.layers[i].trainable = False

In [0]:
model.compile(optimizer = optimizers.Adam(lr=1e-4), loss= 'binary_crossentropy',
              metrics = ['accuracy'])

In [21]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
        '/content/alignedGender/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')
validate_generator = train_datagen.flow_from_directory(
        '/content/alignedGender/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')


Found 14047 images belonging to 2 classes.
Found 3445 images belonging to 2 classes.


In [22]:
history = model.fit_generator(train_generator, steps_per_epoch=440, epochs=4, verbose=1, callbacks=None, validation_data=validate_generator, validation_steps=110, validation_freq=1, class_weight=None, max_queue_size=10, workers=-1, use_multiprocessing=True, shuffle=False)

Epoch 1/4
440/440 [==============================] - 259s 589ms/step - loss: 8406.2803 - acc: 0.7621 - val_loss: 130.9474 - val_acc: 0.7555
Epoch 2/4
440/440 [==============================] - 254s 576ms/step - loss: 130.6202 - acc: 0.8753 - val_loss: 129.7835 - val_acc: 0.8438
Epoch 3/4
440/440 [==============================] - 253s 575ms/step - loss: 129.3122 - acc: 0.9162 - val_loss: 129.2493 - val_acc: 0.8282
Epoch 4/4
440/440 [==============================] - 252s 573ms/step - loss: 128.1365 - acc: 0.9354 - val_loss: 127.6305 - val_acc: 0.8621
